In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import GridSearchCV
from tsfresh.feature_extraction import feature_calculators
#============
%matplotlib inline

from sklearn.svm import NuSVR, SVR
pd.options.display.precision = 8

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from scipy.fftpack import fft, ifft
#=============
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# reading the data in chucks to avoid memory issues
csv = pd.read_csv('../input/train.csv', chunksize=15000000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [ ]:
%%time
# preparing the final train dataframe from chunks
train = pd.DataFrame()
for chunk in csv:
        train = pd.concat([train, chunk], ignore_index=True)

In [ ]:
del csv
gc.collect()
train.head()

In [ ]:
train.tail()

In [ ]:
# code to plot and see the trend of the data
fig, ax1 = plt.subplots(figsize=(20, 8))
plt.title("Trends of acoustic_data and time_to_failure of data")
plt.plot(train['acoustic_data'].values[::50], color='#ef4fa6')
ax1.set_ylabel('acoustic_data', color='#ef4fa6')
plt.legend(['acoustic_data'])
ax2 = ax1.twinx()
plt.plot(train['time_to_failure'].values[::50], color='#2196f3')
ax2.set_ylabel('time_to_failure', color='#2196f3')
plt.legend(['time_to_failure'], loc=(0.9, 0.9))
plt.grid(False)
gc.collect()

In [ ]:
gc.collect()

In [ ]:
#index_ocur = train[train['time_to_failure'].diff(-1)<0].index
#print(index_ocur)

# Assigning the output failure indices to save time
index_ocur = [5656574,  50085878, 104677356, 138772453, 187641820, 218652630,
                245829585, 307838917, 338276287, 375377848, 419368880, 461811623,
                495800225, 528777115, 585568144, 621985673]

In [ ]:
# code to plot and see the trend closley 
fig, ax1 = plt.subplots(figsize=(20, 8))
plt.title("Trends of acoustic_data and time_to_failure of data")
plt.plot(train['acoustic_data'].values[:6300000], color='#ef4fa6')
ax1.set_ylabel('acoustic_data', color='#ef4fa6')
plt.legend(['acoustic_data'])
ax2 = ax1.twinx()
plt.plot(train['time_to_failure'].values[:6300000], color='#2196f3')
ax2.set_ylabel('time_to_failure', color='#2196f3')
plt.legend(['time_to_failure'], loc=(0.9, 0.9))
plt.grid(False)
gc.collect()

In [ ]:
f, axes = plt.subplots(4, 4, figsize=(14, 8))
i = 0
j = 0

for idx in index_ocur:
    ad = train['acoustic_data'].values[idx - 2000000: idx + 30000]
    ttf = train['time_to_failure'].values[idx - 2000000: idx + 30000]
    axes[j][i].plot(ad, color='#ef4fa6')
    axes[j][i].set_xticklabels([])
    s = axes[j][i].twinx()
    s.plot(ttf, color='#2196f3')
    i += 1
    if i >= 4:
        i = 0
        j += 1

plt.tight_layout()
plt.show()
del ad, ttf
gc.collect()

In [ ]:
from sklearn.linear_model import LinearRegression
gc.collect()
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

from scipy import stats

# Create a training file with simple derived features

rows = 150_000
segments = int(np.floor(train.shape[0] / rows))

X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min','q95','q99', 'q05','q01','q03','q97',
                               'abs_max', 'abs_mean', 'abs_std', 'trend', 'abs_trend', 'iqr', 
                                'q999','q001','ave10'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = pd.Series(seg['acoustic_data'].values)
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()
    X_train.loc[segment, 'q95'] = np.quantile(x,0.95)
    X_train.loc[segment, 'q99'] = np.quantile(x,0.99)
    X_train.loc[segment, 'q05'] = np.quantile(x,0.05)
    X_train.loc[segment, 'q01'] = np.quantile(x,0.01)
    X_train.loc[segment, 'q03'] = np.quantile(x,0.03)
    X_train.loc[segment, 'q97'] = np.quantile(x,0.97)
    
    X_train.loc[segment, 'abs_max'] = np.abs(x).max()
    X_train.loc[segment, 'abs_mean'] = np.abs(x).mean()
    X_train.loc[segment, 'abs_std'] = np.abs(x).std()
    X_train.loc[segment, 'trend'] = add_trend_feature(x)
    X_train.loc[segment, 'abs_trend'] = add_trend_feature(x, abs_values=True)
    
    X_train.loc[segment, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_train.loc[segment, 'q999'] = np.quantile(x,0.999)
    X_train.loc[segment, 'q001'] = np.quantile(x,0.001)
    X_train.loc[segment, 'ave10'] = stats.trim_mean(x, 0.1)
    
    for windows in [10, 50, 100, 200, 1000]:
                x_roll_std = x.rolling(windows).std().dropna().values
                x_roll_mean = x.rolling(windows).mean().dropna().values

                X_train.loc[segment, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
                X_train.loc[segment, 'std_roll_std_' + str(windows)] = x_roll_std.std()
                X_train.loc[segment, 'max_roll_std_' + str(windows)] = x_roll_std.max()
                X_train.loc[segment, 'min_roll_std_' + str(windows)] = x_roll_std.min()
                X_train.loc[segment, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
                X_train.loc[segment, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
                X_train.loc[segment, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
                X_train.loc[segment, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
                X_train.loc[segment, 'q03_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.03)
                X_train.loc[segment, 'q97_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.97)
                X_train.loc[segment, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
                X_train.loc[segment, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
                X_train.loc[segment, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()

                X_train.loc[segment, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
                X_train.loc[segment, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
                X_train.loc[segment, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
                X_train.loc[segment, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
                X_train.loc[segment, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
                X_train.loc[segment, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
                X_train.loc[segment, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
                X_train.loc[segment, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
                X_train.loc[segment, 'q03_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.03)
                X_train.loc[segment, 'q97_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.97)
                X_train.loc[segment, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
                X_train.loc[segment, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
                X_train.loc[segment, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
                
            
X_train = X_train[:-10]
y_train = y_train[:-10]

In [ ]:
means_dict = {}
for col in X_train.columns:
    if X_train[col].isnull().any():
        print(col)
        mean_value = X_train.loc[X_train[col] != -np.inf, col].mean()
        X_train.loc[X_train[col] == -np.inf, col] = mean_value
        X_train[col] = X_train[col].fillna(mean_value)
        means_dict[col] = mean_value

In [ ]:
del train
gc.collect()

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)

In [ ]:
gc.collect()
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)
for seg_id in X_test.index:
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = pd.Series(seg['acoustic_data'].values)
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
    X_test.loc[seg_id, 'q95'] = np.quantile(x,0.95)
    X_test.loc[seg_id, 'q99'] = np.quantile(x,0.99)
    X_test.loc[seg_id, 'q05'] = np.quantile(x,0.05)
    X_test.loc[seg_id, 'q01'] = np.quantile(x,0.01)
    X_test.loc[seg_id, 'q03'] = np.quantile(x,0.03)
    X_test.loc[seg_id, 'q97'] = np.quantile(x,0.97)
    
    X_test.loc[seg_id, 'abs_max'] = np.abs(x).max()
    X_test.loc[seg_id, 'abs_mean'] = np.abs(x).mean()
    X_test.loc[seg_id, 'abs_std'] = np.abs(x).std()
    X_test.loc[seg_id, 'trend'] = add_trend_feature(x)
    X_test.loc[seg_id, 'abs_trend'] = add_trend_feature(x, abs_values=True)
    
    X_test.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_test.loc[seg_id, 'q999'] = np.quantile(x,0.999)
    X_test.loc[seg_id, 'q001'] = np.quantile(x,0.001)
    X_test.loc[seg_id, 'ave10'] = stats.trim_mean(x, 0.1)
    
    for windows in [10, 50, 100, 200, 1000]:
                x_roll_std = x.rolling(windows).std().dropna().values
                x_roll_mean = x.rolling(windows).mean().dropna().values

                X_test.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
                X_test.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
                X_test.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
                X_test.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
                X_test.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
                X_test.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
                X_test.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
                X_test.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
                X_test.loc[seg_id, 'q03_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.03)
                X_test.loc[seg_id, 'q97_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.97)
                X_test.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
                X_test.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
                X_test.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()

                X_test.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
                X_test.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
                X_test.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
                X_test.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
                X_test.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
                X_test.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
                X_test.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
                X_test.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
                X_test.loc[seg_id, 'q03_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.03)
                X_test.loc[seg_id, 'q97_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.97)
                X_test.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
                X_test.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
                X_test.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
                
                


In [ ]:
gc.collect()
X_test.isnull().sum().sum()
for col in X_test.columns:
    if X_test[col].isnull().any():
        X_test.loc[X_test[col] == -np.inf, col] = means_dict[col]
        X_test[col] = X_test[col].fillna(means_dict[col])
        
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)


In [ ]:
def train_model(X=X_train_scaled, X_test=X_test_scaled, y=y_train, params=None, folds=folds, model_type='lgb', plot_feature_importance=False, model=None):

    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 50000, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='mae',
                    verbose=10000, early_stopping_rounds=200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_absolute_error(y_valid, y_pred_valid)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
            print('')
            y_pred = model.predict(X_test).reshape(-1,)
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric='MAE', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_valid, y_pred_valid))

        prediction += y_pred    
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
            return oof, prediction, feature_importance
        return oof, prediction
    
    else:
        return oof, prediction
     

In [ ]:
gc.collect()
params = {'num_leaves': 128,
          'min_data_in_leaf': 79,
          'objective': 'huber',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting": "gbdt",
          "bagging_freq": 5,
          "bagging_fraction": 0.8126672064208567,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1302650970728192,
          'reg_lambda': 0.3603427518866501
         }
oof_lgb, prediction_lgb, feature_importance = train_model(params=params, model_type='lgb', plot_feature_importance=True)

In [ ]:
xgb_params = {'eta': 0.03,
              'max_depth': 9,
              'subsample': 0.9,
              'objective': 'reg:linear',
              'eval_metric': 'mae',
              'silent': True,
              'nthread': 4}
oof_xgb, prediction_xgb = train_model(X=X_train_scaled, X_test=X_test_scaled, params=xgb_params, model_type='xgb')

In [ ]:
gc.collect()
submission['time_to_failure'] = (prediction_lgb+prediction_xgb)/2#(prediction_lgb + prediction_xgb + prediction_svr + prediction_svr1 + prediction_cat + prediction_r) / 5
# submission['time_to_failure'] = prediction_lgb_stack
print(submission.head())
submission.to_csv('submission.csv')